<a href="https://colab.research.google.com/github/simonskyjumpp/bigdataanalysis/blob/main/Twitter_data_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!pip install -q kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("cosmos98/twitter-and-reddit-sentimental-analysis-dataset")
print("Path to dataset files:", path)

100%|██████████| 10.0M/10.0M [00:00<00:00, 55.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset/versions/1


In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession
# Start Spark Session
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

In [ ]:
# Create the sample DataFrame
df = spark.read.csv(f"{path}/", header=True, inferSchema=True)
df.show(5)  # Show the first 5 rows
df.printSchema() # Print the schema (column names and data types)

+--------------------+--------+
|          clean_text|category|
+--------------------+--------+
|when modi promise...|      -1|
|talk all the nons...|       0|
|what did just say...|       1|
|asking his suppor...|       1|
|answer who among ...|       1|
+--------------------+--------+
only showing top 5 rows

root
 |-- clean_text: string (nullable = true)
 |-- category: string (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import length
from pyspark.ml.feature import Imputer


In [ ]:
if 'sentiment' in df.columns:
    df = df.na.drop(subset=['clean_text', 'category'])

In [ ]:
# Index the 'category' column
indexer = StringIndexer(inputCol="category", outputCol="label", handleInvalid="skip" )
indexed_df = indexer.fit(df).transform(df)
indexed_df = indexed_df.withColumn("text_length", length(indexed_df["clean_text"]))

In [ ]:
imputer = Imputer(
    inputCols=["text_length"],
    outputCols=["text_length_imputed"]
).setStrategy("mean")

In [ ]:
indexed_df = imputer.fit(indexed_df).transform(indexed_df)

In [ ]:
# Create the features vector
assembler = VectorAssembler(inputCols=["text_length_imputed"], outputCol="features")
df_with_features = assembler.transform(indexed_df)

In [ ]:
# Train a Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(df_with_features)

In [ ]:
# Make predictions
predictions = model.transform(df_with_features)
# Show some predictions
predictions.select("clean_text", "category", "label", "prediction").show(50, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----+----------+
|clean_text                                                                                                                                                                                                                                  |category|label|prediction|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+-----+----------+
|when modi promised “minimum government maximum governance” expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and t

In [ ]:
# Stop Spark session
spark.stop()